In [1]:
%matplotlib widget

import os  
import sys  
sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
import pandas as pd
from amftrack.util.sys import get_dates_datetime, get_dirname, get_plate_number, get_postion_number

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import from_sparse_to_graph, generate_nx_graph, sparse_to_doc
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import run_parallel, find_state, directory_scratch, directory_project, path_code

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests


In [5]:
inst_25 = [(9,0,11),(9,13,35),(3,0,19),(37,0,8),(11,0,30),(19,0,25),(13,0,25),(39,0,18),(35,0,15),(29,0,20)]
inst_bait = [(10,0,10), (14,0,11), (33,0,26),(4,2,18),(4,20,30),(39,117,137),(12,5,21),(28,0,14),(32,5,14),(32,15,44),(36,0,9),(40,0,14),(2,1,15),(2,16,35),(5,160,168),(11,158,164),(13,116,131)]
inst_30 = []
inst_25late = []
treatments = {'25*' : inst_25late,'25' : inst_25,'baits' : inst_bait, '30' : inst_30}
plate_number = {(9,0,11) : 296, (9,13,35) : 296, (3,0,19) : 340, (37,0,8) : 269,(11,0,30) : 314, (19,0,25) : 344, (13,0,25) : 298, (39,0,18) : 297, (35,0,15) : 351,(10,0,10) : 395,(14,0,11) : 399, (33,0,26) : 420, (4,2,18) : 423, (4,20,30) : 423,(8,0,17): 434 ,(8,20,30) : 434,(39,117,137) : 433, (12,5,21) : 436, (28,0,14): 405,(32,5,45):409,(36,0,9) : 419,(40,0,14) : 425,(2,1,15):435,(2,16,35):435,(5,160,168):382,(11,158,164) : 416,(13,116,131) : 424, (29,0,20) : 373,(32,15,44):409, (32,5,14) : 409, (40,153,153+37) : 69}

In [2]:
results={}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(open(f'{path_code}/MscThesis/Results/bas_{inst}.pick', "rb"))

FileNotFoundError: [Errno 2] No such file or directory: '/home/cbisot/pycode//MscThesis/Results/bas_(38, 7, 42).pick'

In [3]:
column_names = ["plate","inst", "treatment", "bas_frequ"]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        bas_frequs = results[inst]
        new_line = pd.DataFrame(
            {   "plate": [plate_number[inst]],
                "inst": [inst],
                "treatment": [treatment],
                "bas_frequ": [np.mean(bas_frequs)],
            }
        )  # index 0 for
        # mothers need to be modified to resolve multi mother issue
        infos = infos.append(new_line, ignore_index=True)

KeyError: (38, 7, 42)

In [4]:
infos

,plate,inst,treatment,bas_frequ
0,69,"(40, 153, 190)",25*,8.467117
1,94,"(39, 269, 329)",25*,9.227986
2,102,"(40, 262, 287)",25*,7.214128


In [9]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = infos.boxplot(column = ['bas_frequ'],by="treatment",figsize =(9,8),ax =ax,patch_artist=True,notch=True,showmeans = True)
colors = ['lightblue'] + ['pink']+['lightgreen']
for i,(artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col) 
ax.set_xlabel('Treatment',size=15)
ax.set_ylabel('BAS frequency ($BAS.cm^{-1}$)',size=15)
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [7]:
ztest ,pval1 = stests.ztest(infos.loc[(infos["treatment"] == "25")]['bas_frequ'],infos.loc[infos["treatment"] == "baits"]['bas_frequ'], value=0,alternative='two-sided')
pval1,ztest

(0.852274623957826, 0.18621696058726883)

In [6]:
mapping = {298: 0,94 : 0, 69:0, 420 : 2, 433 : 2, 399:2, 42: 1, 48: 1,60:1}
key = infos['plate'].map(mapping)

In [7]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = infos.plot.bar(x="plate", y='bas_frequ',figsize =(9,8),ax =ax, color = ['lightblue']*3  + ['pink']*3 +['lightgreen']*3,legend=False)
# colors = ['lightblue'] + ['pink']+['lightgreen']
# for i,(artist, col) in enumerate(zip(ax.artists, colors)):
#     artist.set_edgecolor(col)
#     artist.set_facecolor(col) 
ax.set_xlabel('Plate number',size=15)
ax.set_ylabel('BAS frequency ($BAS.cm^{-1}$)',size=15)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
ztest ,pval1 = stests.ztest(infos.loc[infos["treatment"] == "25"]['bas_frequ'],infos.loc[infos["treatment"] == "30"]['bas_frequ'], value=0,alternative='two-sided')
pval1

0.6621235504678156

In [4]:
bas_frequ,bass = estimate_bas_freq((24,104,116),200,0,criter)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(bas_frequ),max(2,-int(np.log10(np.nanmean(bas_frequ)))+2))}, std = {round(np.nanstd(bas_frequ),max(2,-int(np.log10(np.nanstd(bas_frequ)))+2))}')

ax.hist(np.array(bas_frequ),10,density=True)
np.mean(bas_frequ),np.std(bas_frequ)

begin = 2020-12-08 10:57:00 
  end = 2020-12-10 11:07:00


/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(5.463183153628295, 1.610400934218557)

In [5]:
inst_25 = [(13,0,19),(39,404,464),(40,0,37)]
bas_frequs_25 = estimate_bas_freq_mult(inst_25,1000,0,criter)
bas_frequ_25=[np.mean(bas_frequ) for bas_frequ in bas_frequs_25]

begin = 2020-07-01 15:57:00 
  end = 2020-07-04 19:57:00
begin = 2020-11-23 15:01:00 
  end = 2020-11-26 03:02:00
begin = 2020-11-19 20:21:00 
  end = 2020-11-25 18:06:00


In [8]:
pickle.dump(bas_frequ_25, open(f'{path_code}/MscThesis/Results/bas_{treatment}.pick', "wb"))

In [24]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(bas_frequ_25),max(2,-int(np.log10(np.nanmean(bas_frequ_25)))+2))}, std = {round(np.nanstd(bas_frequ_25),max(2,-int(np.log10(np.nanstd(bas_frequ_25)))+2))}')

ax.hist(np.array(bas_frequ_25),10,density=True)
np.mean(bas_frequ_25),np.std(bas_frequ_25)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(9.426201243726451, 0.2715255985498475)

In [ ]:
treatment = 'baits'
inst_bait = treatments[treatment]
bas_frequs_bait = estimate_bas_freq_mult(inst_bait,1000,0,criter)
bas_frequ_bait=[np.mean(bas_frequ) for bas_frequ in bas_frequs_bait]

begin = 2020-09-29 15:17:00 
  end = 2020-10-03 19:34:00


In [ ]:
pickle.dump(bas_frequ_bait, open(f'{path_code}/MscThesis/Results/bas_{treatment}.pick', "wb"))

In [21]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(bas_frequ_bait),max(2,-int(np.log10(np.nanmean(bas_frequ_bait)))+2))}, std = {round(np.nanstd(bas_frequ_bait),max(2,-int(np.log10(np.nanstd(bas_frequ_bait)))+2))}')

ax.hist(np.array(bas_frequ_bait),10,density=True)
np.mean(bas_frequ_bait),np.std(bas_frequ_bait)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(13.004325494685883, 2.864767750796865)

In [ ]:
inst_30 = [(26,30,53),(22,8,26),(24,105,110)]
bas_frequs_30 = estimate_bas_freq_mult(inst_bait,1000,0,criter)
bas_frequ_30=[np.mean(bas_frequ) for bas_frequ in bas_frequs_30]

begin = 2020-09-29 15:17:00 
  end = 2020-10-03 19:34:00


In [9]:
plt.close('all')
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(bas_frequ_30),max(2,-int(np.log10(np.nanmean(bas_frequ_30)))+2))}, std = {round(np.nanstd(bas_frequ_30),max(2,-int(np.log10(np.nanstd(bas_frequ_30)))+2))}')

ax.hist(np.array(bas_frequ_30),10,density=True)
np.mean(bas_frequ_30),np.std(bas_frequ_30)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(13.049226448227035, 2.8916423120372134)

In [19]:
from scipy import stats
from statsmodels.stats import weightstats as stests
ztest ,pval1 = stests.ztest(bas_frequ_25, [frequ +0.1 for frequ in bas_frequ_25], value=0,alternative='two-sided')
print(float(pval1))

0.04952564573481898


In [11]:
pval1

0.0

In [13]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
# ax.set_title(f'mean = {round(np.nanmean(bas_frequ),max(2,-int(np.log10(np.nanmean(bas_frequ)))+2))}, std = {round(np.nanstd(bas_frequ),max(2,-int(np.log10(np.nanstd(bas_frequ)))+2))}')

ax.hist(np.array(bas_frequ_25),30,density=True)
np.mean(bas_frequ_25),np.std(bas_frequ_25)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(9.440654195896956, 1.8183353721940376)

In [14]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.set_title(f'mean = {round(np.nanmean(bas_frequ_bait),max(2,-int(np.log10(np.nanmean(bas_frequ_bait)))+2))}, std = {round(np.nanstd(bas_frequ_bait),max(2,-int(np.log10(np.nanstd(bas_frequ_bait)))+2))}')

ax.hist(np.array(bas_frequ_bait),30,density=True)
np.mean(bas_frequ_bait),np.std(bas_frequ_bait)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(13.02672488187662, 2.8492445230602517)

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ),10,density=True)

In [ ]:
np.mean(bas_frequ),np.std(bas_frequ)

In [ ]:
plt.close('all')
bas_frequ = estimate_bas_freq(exp3,200,19,thresh_growth=100,thresh_length=2000)

In [ ]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ),10,density=True)

In [ ]:
np.mean(bas_frequ),np.std(bas_frequ)

In [6]:
inst = 33,0,26
plate = inst[0]
begin = inst[1]
end = inst[2]
dates_datetime = get_dates_datetime(directory,plate)
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = dates_datetime_chosen
exp = pickle.load( open(f'{directory}/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

begin = 2020-09-28 16:15:00 
  end = 2020-10-05 11:00:00


In [7]:
select_hyph = {}
for hyph in exp.hyphaes:
    select_hyph[hyph] = []
    for i,t in enumerate(hyph.ts[:-1]):
        tp1=hyph.ts[i+1]
        pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
        length = np.sum([get_length_um(seg) for seg in pixels])/get_time(exp,t,tp1)
        select_hyph[hyph].append((t,hyph.ts[i+1],length,pixels,nodes))

In [8]:
max_growths = []
lengths = []
branch_frequ = []
hyph_l = []
RH=[]
BAS=[]
thresh_growth = 100
thresh_length = 2000
hyph_info = {}
for hyph in exp.hyphaes:
    growths = [c[2] for c in select_hyph[hyph]]
    ts = [c[0] for c in select_hyph[hyph]]
    tp1s = [c[1] for c in select_hyph[hyph]]
    if len(growths)>0:
        length = hyph.get_length_um(hyph.ts[-1])
        nodes = hyph.get_nodes_within(hyph.ts[-1])[0]
        max_growth = np.max(growths)
        total_growth = np.sum([growth * get_time(exp,ts[i],tp1s[i]) for i,growth in enumerate(growths)])
        if max_growth>=thresh_growth  and length>=thresh_length:
            RH.append(hyph)
        else:
            BAS.append(hyph)
        lengths.append(length)
        max_growths.append(max_growth)
        branch_frequ.append((len(nodes)-1)/(length+1))
        hyph_l.append(hyph)
        hyph_info[hyph]={'max_growth':max_growth,'length_total':length,'total_growth':total_growth,'nodes':nodes}

In [9]:
last_t = len(exp.nx_graph)-1
node1 = Node(choice(list(exp.nx_graph[last_t].nodes)),exp)
node2 = Node(choice(list(exp.nx_graph[last_t].nodes)),exp)

In [11]:
node1,node2

(Node(46657), Node(43667))

In [14]:
Node(36431,exp).degree(last_t)

3

In [16]:
exp.dates

[datetime.datetime(2020, 9, 28, 16, 15),
 datetime.datetime(2020, 9, 28, 20, 15),
 datetime.datetime(2020, 9, 29, 0, 16),
 datetime.datetime(2020, 9, 29, 4, 15),
 datetime.datetime(2020, 9, 29, 8, 15),
 datetime.datetime(2020, 9, 29, 15, 17),
 datetime.datetime(2020, 9, 29, 19, 16),
 datetime.datetime(2020, 9, 29, 23, 17),
 datetime.datetime(2020, 9, 30, 19, 17),
 datetime.datetime(2020, 9, 30, 23, 17),
 datetime.datetime(2020, 10, 1, 7, 17),
 datetime.datetime(2020, 10, 1, 11, 35),
 datetime.datetime(2020, 10, 1, 15, 34),
 datetime.datetime(2020, 10, 1, 19, 34),
 datetime.datetime(2020, 10, 1, 23, 34),
 datetime.datetime(2020, 10, 2, 3, 34),
 datetime.datetime(2020, 10, 2, 7, 34),
 datetime.datetime(2020, 10, 2, 11, 34),
 datetime.datetime(2020, 10, 2, 15, 34),
 datetime.datetime(2020, 10, 2, 19, 34),
 datetime.datetime(2020, 10, 2, 23, 34),
 datetime.datetime(2020, 10, 3, 3, 34),
 datetime.datetime(2020, 10, 3, 7, 34),
 datetime.datetime(2020, 10, 3, 11, 34),
 datetime.datetime(2020,

In [15]:
plt.close('all')
nodes = nx.shortest_path(exp.nx_graph[-1], source = node1.label, target = node2.label)
exp.plot([last_t],[nodes])

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [67]:
last_t = len(exp.nx_graph)-1
bass=[]
for node in nodes:
    for hyph in exp.hyphaes:
        if exp.get_node(node)==hyph.root:
            if hyph in BAS:
                bass.append(hyph)
                hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
len(bass)/get_length_um_node_list(nodes,exp,last_t)*10000


9.18428587716088

In [ ]:
bas_frequ = estimate_bas_freq(exp,samples,t,thresh_growth=150,thresh_length=1000)

In [ ]:
edge = choice(list(exp.nx_graph[-1].edges))

In [21]:
for bas in BAS:
    result = bas.get_nodes_within(bas.ts[-1])
    break

In [ ]:
edge

In [56]:
bas_frequ=[]
for rh in RH:
    bass=[]
    for node in rh.get_nodes_within(rh.ts[-1])[0][1:-1]:
        for hyph in exp.hyphaes:
            if exp.get_node(node)==hyph.root:
                if hyph in BAS:
                    bass.append(hyph)
                    break
    #             print(hyph)
    #             if hyph.ts[0] in rh.ts:
#                 print(hyph,hyph_info[hyph],hyph in BAS)
#                 hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])
    bas_frequ.append(len(bass)/hyph_info[rh]['length_total'])
            

In [57]:
bas_frequ

[0.0009122727654370715,
 0.00028670026287244547,
 0.0009578442600032728,
 0.0013020471611031223,
 0.001326741715756687,
 0.0003003334594429773,
 0.0006402714171057274,
 0.0009102818532015862,
 0.0011218383112101356,
 0.0007882483743865315,
 0.0016954735999182588,
 0.0,
 0.000912836307092113,
 0.000247549378106453,
 0.00022801921308447938,
 0.0008117649416916064,
 0.0027472228445821175,
 0.0,
 0.0022354396266294685,
 0.0005645697310763812,
 0.0008549715308009903,
 0.0,
 0.0006072330900587639,
 0.0016995668275699184,
 0.0]

In [58]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.set_xlabel('bas_frequ (bas/mm)')
ax.set_ylabel('density')
# ax.set_xlim((0,2))
ax.hist(np.array(bas_frequ)*10000,10,density=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([0.08736095, 0.02912032, 0.07280079, 0.07280079, 0.04368048,
        0.        , 0.02912032, 0.        , 0.01456016, 0.01456016]),
 array([ 0.        ,  2.74722284,  5.49444569,  8.24166853, 10.98889138,
        13.73611422, 16.48333707, 19.23055991, 21.97778276, 24.7250056 ,
        27.47222845]),
 <a list of 10 Patch objects>)

In [8]:
len(RH)

126

In [174]:
rh = choice(RH)
rh.ts

[27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37]

In [175]:
plt.close('all')
t0 = select_hyph[rh][0][0]
times=[get_time(exp,t0,c[0]) for c in select_hyph[rh]]
speeds = [c[2] for c in select_hyph[rh]]
plt.close('all')
fig=plt.figure(figsize=(8,8))
ax = fig.add_subplot(111)
ax.set_xlabel('time (hour)')
ax.set_ylabel('speed (um/hour)')
ax.plot(times,speeds)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [176]:
rh.get_nodes_within(rh.ts[-1])[0]

[9110, 9873, 9865, 9855, 9109]

In [177]:
plt.close('all')
exp.plot(rh.ts,[rh.get_nodes_within(rh.ts[-1])[0]+[]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [178]:
plt.close('all')
for node in rh.get_nodes_within(rh.ts[-1])[0][1:-1]:
    for hyph in exp.hyphaes:
        if exp.get_node(node)==hyph.root:
#             print(hyph)
#             if hyph.ts[0] in rh.ts:
            print(hyph,hyph_info[hyph],hyph in BAS)
            hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])

(Node(9862), Node(9873)) (68.55772085948638, 432.47828936436974, 148.6429469009527, [9873, 14704, 9862]) True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(Node(11846), Node(9865)) (0.0, 139.72161256677649, 0.0, [9865, 11165, 11846]) True


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(7.646535511603008, 5.143899884825599)

In [25]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(8.98550841574617, 4.786839847744506)

In [9]:
np.mean(bas_frequ)*10000,np.std(bas_frequ)*10000

(4.84338132339359, 3.625468207834553)

In [2]:
15*4096*1.725

105984.0

In [92]:
plt.close('all')
exp.plot(rh.ts,[[rh.end.label,rh.root.label]]*len(rh.ts))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
rh.end.ts()

[16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28]

In [159]:
rh.end.show_source_image(rh.ts[-1],rh.ts[-1]+1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [40]:
edge

9823

In [45]:
edge = choice(list(exp.nx_graph[-1].edges))
edge_obj =Edge(Node(edge[0],exp),Node(edge[1],exp),exp)

In [49]:
edge[0]

998